In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read in the Data

In [ ]:
import pandas as pd
import glob
import os
import numpy as np

# Full NCAA Dataset - 48 sec
df = pd.read_csv("/content/drive/MyDrive/USC Baseball/Datasets/Updated22_23_NCAA_Dataset.csv")

<ipython-input-2-7e143f33715e>:7: DtypeWarning: Columns (27,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/USC Baseball/Datasets/Updated22_23_NCAA_Dataset.csv")


# Clean up the data + Create Buckets


In [ ]:
#This will ignore all DeprecationWarning warnings in your code.
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
cleandf = df[df['TaggedPitchType'].str.contains("Undefined|Other|,") == False]
cleanD1 = cleandf[cleandf['Level'] == "D1"]

In [ ]:
cleanConf = cleanD1[cleanD1['League'].isin(["SEC", "BIG12", "BIG10", "ACC", "PAC12", "CUSA", "SBELT", "AMER"])]

In [ ]:
# Reclassifies pitch type buckets
conditions = [
    (cleanConf['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker')),  # Fastballs
    (cleanConf['TaggedPitchType'].str.contains('Slider|Cutter|Curveball')),  # Breaking Balls
    (cleanConf['TaggedPitchType'].str.contains('ChangeUp|Splitter|Knuckleball')),  # Offspeed
]

# creates a list of values
values = [1, 2, 3]
# creates new column determining hit or not and assign values to it based on the conditions
cleanConf['PitchBucket'] = np.select(conditions, values, default = 4)

<ipython-input-6-7b1c0ce79f90>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['PitchBucket'] = np.select(conditions, values, default = 4)


In [ ]:
cleanConf['PitchBucket'].value_counts()

,count
PitchBucket,
1,1001439
2,576786
3,213075


In [ ]:
# Recodes Pitch Handedness to binary variable

conditions = [
    (cleanConf['PitcherThrows'] == 'Right'),  # RHP
    (cleanConf['PitcherThrows'] == 'Left')  # LHP
]

# creates a list of values to assign for pitch handedness where RHP = 1 & LHP = 0
values = [0, 1]
# creates new column determining pitch hand
cleanConf['PitcherHand'] = np.select(conditions, values)

<ipython-input-8-783810b04da9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['PitcherHand'] = np.select(conditions, values)


In [ ]:
cleanConf['PitcherHand'].value_counts()

,count
PitcherHand,
0,1290893
1,500407


In [ ]:
conditions = [
    (cleanConf['PitchCall'] != 'StrikeSwinging'),  # Whiff
    (cleanConf['PitchCall'] == 'StrikeSwinging')  # Not Whiff
]

# creates a list of values to assign for pitch handedness where RHP = 1 & LHP = 0
values = [0, 1]
# creates new column determining pitch hand
cleanConf['isWhiff'] = np.select(conditions, values)

<ipython-input-10-4888637904f7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['isWhiff'] = np.select(conditions, values)


In [ ]:
cleanConf['isWhiff'].value_counts()

,count
isWhiff,
0,1599860
1,191440


In [ ]:
cleanConf['SpinAxisRad'] = cleanConf['SpinAxis'] * (np.pi / 180)

<ipython-input-12-d40f482f83ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['SpinAxisRad'] = cleanConf['SpinAxis'] * (np.pi / 180)


In [ ]:
cleanConf['TSM'] = cleanConf['SpinRate']*np.sin(cleanConf['SpinAxisRad'])
cleanConf['G'] = cleanConf['SpinRate']*np.cos(cleanConf['SpinAxisRad'])
cleanConf['SpinEfficiency'] = ((cleanConf['TSM']+cleanConf['G'])/cleanConf['SpinRate'])*100

<ipython-input-25-e3435536d871>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['TSM'] = cleanConf['SpinRate']*np.sin(cleanConf['SpinAxisRad'])
<ipython-input-25-e3435536d871>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['G'] = cleanConf['SpinRate']*np.cos(cleanConf['SpinAxisRad'])
<ipython-input-25-e3435536d871>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [ ]:
print(cleanConf['SpinEfficiency'].describe())

count    1.777704e+06
mean    -4.788563e+01
std      9.928350e+01
min     -1.414214e+02
25%     -1.353242e+02
50%     -9.647552e+01
75%      3.082508e+01
max      1.414214e+02
Name: SpinEfficiency, dtype: float64


In [ ]:
cleanConf['NormalizedSpinEfficiency'] = (np.abs(cleanConf['SpinEfficiency']))

<ipython-input-27-517187e8e63a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['NormalizedSpinEfficiency'] = (np.abs(cleanConf['SpinEfficiency']))


In [ ]:
print(cleanConf['NormalizedSpinEfficiency'].describe())

count    1.777704e+06
mean     1.006566e+02
std      4.492762e+01
min      2.221441e-04
25%      6.448470e+01
50%      1.234364e+02
75%      1.375631e+02
max      1.414214e+02
Name: NormalizedSpinEfficiency, dtype: float64


In [ ]:
spinColf = cleanConf[['TaggedPitchType', 'RelSpeed', 'SpinRate', 'SpinAxis', 'InducedVertBreak', 'HorzBreak', 'SpinAxisRad', 'TSM', 'G', 'SpinEfficiency']]

In [ ]:
spinColf.head()

,TaggedPitchType,RelSpeed,SpinRate,SpinAxis,InducedVertBreak,HorzBreak,SpinAxisRad,TSM,G,SpinEfficiency
0,Fastball,99.52724,2694.881660,228.580072,16.74166,17.86536,3.989475,-2020.840592,-1782.860192,-141.145373
1,Fastball,99.97207,2591.952252,230.378430,14.54879,16.43906,4.020862,-1996.511404,-1652.924285,-140.798724
2,Curveball,82.98675,2868.547909,61.314872,-3.63477,-9.01126,1.070146,2516.493272,1376.890961,135.726659
3,Slider,86.45521,2809.630437,126.013235,7.70886,-9.07776,2.199346,2272.657233,-1651.984350,22.090908
4,Curveball,82.26110,2939.165899,60.441938,-5.50744,-12.05288,1.054911,2556.651799,1449.906122,136.316154


In [ ]:
swingDataset = cleanConf[cleanConf['PitchCall'].isin(['InPlay', 'StrikeSwinging', 'FoulBall', 'FoulBallNotFieldable'])]

In [ ]:
print("Clean Conference size: " + str(len(cleanConf)))
print("Swing Dataset size: " + str(len(swingDataset)))

Clean Conference size: 1791300
Swing Dataset size: 774807


# Determine Important Variables

Determine if multicollinearity exists within pitching variables


In [ ]:
multiCollinDF = cleanConf[['isWhiff', 'PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'ZoneSpeed', 'EffectiveVelo', 'SpeedDrop', 'pfxx', 'pfxz', 'VertApprAngle', 'HorzApprAngle']].dropna()
multiCollinDF.corr()

,isWhiff,PitchBucket,PitcherHand,RelSpeed,SpinRate,InducedVertBreak,HorzBreak,SpinAxis,RelHeight,RelSide,Extension,ZoneSpeed,EffectiveVelo,SpeedDrop,pfxx,pfxz,VertApprAngle,HorzApprAngle
isWhiff,1.000000,0.093343,0.003586,-0.039120,0.014628,-0.042654,-0.022979,-0.018408,0.001544,-0.004036,-0.012853,-0.037220,-0.038924,-0.032847,0.021934,-0.039482,-0.063317,-0.016779
PitchBucket,0.093343,1.000000,0.004624,-0.695334,-0.245902,-0.547556,-0.134422,-0.132801,-0.080542,0.007622,-0.179765,-0.676723,-0.680270,-0.508560,0.127693,-0.544023,-0.562087,-0.086414
PitcherHand,0.003586,0.004624,1.000000,-0.116627,-0.086647,0.023906,-0.448525,-0.030696,-0.039442,-0.904429,-0.031161,-0.122078,-0.116357,-0.042724,0.516145,0.018949,-0.004058,0.608695
RelSpeed,-0.039120,-0.695334,-0.116627,1.000000,0.066475,0.716134,0.289880,0.257144,0.151843,0.088253,0.295483,0.988388,0.986808,0.656133,-0.285758,0.722530,0.677450,0.082468
SpinRate,0.014628,-0.245902,-0.086647,0.066475,1.000000,-0.149148,-0.139666,-0.198824,0.024109,0.078591,-0.192318,0.074411,0.036478,0.000371,0.124347,-0.146886,-0.056256,-0.177924
InducedVertBreak,-0.042654,-0.547556,0.023906,0.716134,-0.149148,1.000000,0.210095,0.269118,0.207981,-0.051081,0.339514,0.684607,0.719693,0.585153,-0.196623,0.997211,0.736063,0.170180
HorzBreak,-0.022979,-0.134422,-0.448525,0.289880,-0.139666,0.210095,1.000000,0.795557,-0.024149,0.430431,0.122156,0.282632,0.290051,0.209481,-0.995104,0.211584,0.199722,0.199595
SpinAxis,-0.018408,-0.132801,-0.030696,0.257144,-0.198824,0.269118,0.795557,1.000000,-0.060448,0.052655,0.131352,0.247539,0.260467,0.201590,-0.763328,0.267081,0.237688,0.442085
RelHeight,0.001544,-0.080542,-0.039442,0.151843,0.024109,0.207981,-0.024149,-0.060448,1.000000,-0.055143,-0.006634,0.166514,0.146077,0.018012,0.027350,0.237565,-0.132446,0.030329
RelSide,-0.004036,0.007622,-0.904429,0.088253,0.078591,-0.051081,0.430431,0.052655,-0.055143,1.000000,0.011479,0.091892,0.085355,0.034743,-0.504893,-0.049904,0.010106,-0.686033


In [ ]:
import statistics

# 'target_column' is the name of the column with 0s and 1s, and 'predictor_columns' is a list of predictor column names.

def compute_statistics_by_target(df, target_column, predictor_columns):
    # Group the dataframe by the target variable and calculate statistics for each group
    grouped = df.groupby(target_column)

    # Initialize a dictionary to store the statistics for each group
    statistics_dict = {}

    for target_value, group in grouped:
        # Calculate the means for each predictor column
        means = group[predictor_columns].mean()

        # Calculate the standard deviations for each predictor column
        std_devs = group[predictor_columns].std()

        # Calculate the medians for each predictor column
        medians = group[predictor_columns].median()

        # Calculate the 25th percentile for each predictor column
        percentile_25 = group[predictor_columns].quantile(0.25)

        # Calculate the 75th percentile for each predictor column
        percentile_75 = group[predictor_columns].quantile(0.75)

        # Combine the means and standard deviations for each group into a single row
        row = means.tolist() + std_devs.tolist() + medians.tolist() + percentile_25.tolist() + percentile_75.tolist()

        # Add the row to the statistics dictionary
        statistics_dict[target_value] = row

    # Convert the dictionary to a dataframe and set the index
    statistics_df = pd.DataFrame.from_dict(statistics_dict, orient='index', columns=predictor_columns +
                                        [f'Std_Dev_{col}' for col in predictor_columns] +
                                        [f'Median_{col}' for col in predictor_columns] +
                                        [f'Percentile_25_{col}' for col in predictor_columns] +
                                        [f'Percentile_75_{col}' for col in predictor_columns])


    return statistics_df

In [ ]:
target_column_name = 'isWhiff'
predictor_columns_list = ['PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'ZoneSpeed', 'EffectiveVelo', 'SpeedDrop', 'pfxx', 'pfxz', 'VertApprAngle', 'HorzApprAngle']

fastballBucketDF = cleanD1[cleanD1['PitchBucket'] == 1]
breakingBucketDF = cleanD1[cleanD1['PitchBucket'] == 2]
offspeedBucketDF = cleanD1[cleanD1['PitchBucket'] == 3]

In [ ]:
# Fastball
statistics_table = compute_statistics_by_target(fastballBucketDF, target_column_name, predictor_columns_list)
print(statistics_table)
csv = statistics_table.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+ Colab Notebooks/Stuff_Fastball_WhiffANOVA*.csv", index=False)

   PitchBucket  PitcherHand   RelSpeed     SpinRate  InducedVertBreak  \
0          1.0     0.267692  89.307876  2174.906657         14.880015   
1          1.0     0.279493  89.954842  2203.419123         15.774856   

   HorzBreak    SpinAxis  RelHeight   RelSide  Extension  ...  \
0   5.087537  197.664474   5.741206  0.746418    5.90403  ...   
1   4.475911  195.215676   5.748499  0.700092    5.92399  ...   

   Percentile_75_RelHeight  Percentile_75_RelSide  Percentile_75_Extension  \
0                  6.09734                1.94815                  6.26319   
1                  6.10433                1.94162                  6.27762   

   Percentile_75_ZoneSpeed  Percentile_75_EffectiveVelo  \
0                 83.39630                    90.487818   
1                 83.89188                    91.201358   

   Percentile_75_SpeedDrop  Percentile_75_pfxx  Percentile_75_pfxz  \
0                  8.54495             2.66087            10.59319   
1                  8.69375     

In [ ]:
# Breaking
statistics_table = compute_statistics_by_target(breakingBucketDF, target_column_name, predictor_columns_list)
print(statistics_table)
csv = statistics_table.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+ Colab Notebooks/Stuff_Breaking_WhiffANOVA*.csv", index=False)

   PitchBucket  PitcherHand   RelSpeed     SpinRate  InducedVertBreak  \
0          2.0     0.237368  79.454942  2338.392332         -0.929100   
1          2.0     0.237672  80.455371  2369.676039         -0.282645   

   HorzBreak    SpinAxis  RelHeight   RelSide  Extension  ...  \
0  -3.705261  137.024107   5.681076  0.898038   5.441514  ...   
1  -3.500413  139.993319   5.696293  0.889875   5.466424  ...   

   Percentile_75_RelHeight  Percentile_75_RelSide  Percentile_75_Extension  \
0                 6.031310               2.061890                 5.794680   
1                 6.042445               2.058805                 5.815168   

   Percentile_75_ZoneSpeed  Percentile_75_EffectiveVelo  \
0                75.486510                    80.726615   
1                76.344785                    81.620390   

   Percentile_75_SpeedDrop  Percentile_75_pfxx  Percentile_75_pfxz  \
0                 7.143540             4.62142             2.37316   
1                 7.179032     

In [ ]:
# Offspeed
statistics_table = compute_statistics_by_target(offspeedBucketDF, target_column_name, predictor_columns_list)
print(statistics_table)
csv = statistics_table.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+ Colab Notebooks/Stuff_Offspeed_WhiffANOVA*.csv", index=False)

   PitchBucket  PitcherHand   RelSpeed     SpinRate  InducedVertBreak  \
0          3.0     0.307470  81.073227  1737.286028          7.436943   
1          3.0     0.310547  81.041662  1744.803533          7.301050   

   HorzBreak    SpinAxis  RelHeight   RelSide  Extension  ...  \
0   4.930726  202.540137   5.615679  0.652038   5.841341  ...   
1   5.019775  203.661589   5.669418  0.604446   5.841285  ...   

   Percentile_75_RelHeight  Percentile_75_RelSide  Percentile_75_Extension  \
0                 5.946042                2.02434                  6.22813   
1                 5.991990                1.96549                  6.22648   

   Percentile_75_ZoneSpeed  Percentile_75_EffectiveVelo  \
0                76.375593                    82.512367   
1                76.254140                    82.339000   

   Percentile_75_SpeedDrop  Percentile_75_pfxx  Percentile_75_pfxz  \
0                  7.50340            5.526088             6.57545   
1                  7.49446     

# Making the model

In [ ]:
finalCleanDF = swingDataset.dropna(subset=['isWhiff', 'PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'SpeedDrop', 'pfxz', 'VertApprAngle', 'HorzApprAngle'])
finalDF = finalCleanDF[['isWhiff', 'PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'SpeedDrop', 'pfxz', 'VertApprAngle', 'HorzApprAngle']]

In [ ]:
finalFastball = finalDF[finalDF['PitchBucket'] == 1]
finalBreaking = finalDF[finalDF['PitchBucket'] == 2]
finalOffspeed = finalDF[finalDF['PitchBucket'] == 3]

In [ ]:
print(finalDF.shape)
print(finalFastball.shape)
print(finalBreaking.shape)
print(finalOffspeed.shape)

(768375, 15)
(439313, 15)
(230936, 15)
(98126, 15)


## Making Fastball Model

In [ ]:
minority_class = finalFastball[finalFastball['isWhiff'] == 1]
majority_class = finalFastball[finalFastball['isWhiff'] == 0]

print("majority class is: " + str(len(majority_class)) + " (" + str(100*len(majority_class)/(len(majority_class)+len(minority_class))) + ")")
print("minority class is: " + str(len(minority_class)) + " (" + str(100*len(minority_class)/(len(majority_class)+len(minority_class))) + ")")

print("total is: " + str(len(majority_class)+len(minority_class)))

majority class is: 358670 (81.64338410199562)
minority class is: 80643 (18.356615898004385)
total is: 439313


In [ ]:
# resamples the class
from sklearn.utils import resample
majority_class_downsampled = resample(majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=len(minority_class)*3,  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results
minority_class_upsampled = resample(minority_class,
                                      replace=True,  # sample without replacement
                                      n_samples=int(len(minority_class)*1.25),  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results

# Combine minority class and downsampled majority class
balanced_df = pd.concat([minority_class_upsampled, majority_class_downsampled])




In [ ]:
# Shuffle the data to avoid order effects
# figure why this don't work
balanced_df = balanced_df.sample(frac=1, random_state=42)


In [ ]:
rel_data = balanced_df[['isWhiff', 'PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'SpeedDrop', 'pfxz', 'VertApprAngle', 'HorzApprAngle']].to_numpy()


classification = balanced_df['isWhiff'].to_numpy()

In [ ]:
minority = balanced_df[balanced_df['isWhiff'] == 1]
majority = balanced_df[balanced_df['isWhiff'] != 1]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

sampledTotal = len(majority) + len(minority)
trainTotal = int(0.8*sampledTotal)

majority class is: 241929
minority class is: 100803
total is: 342732


In [ ]:
# Split data into features and target
X = balanced_df.drop(['isWhiff'], axis=1)
y = balanced_df['isWhiff']
# x = np.unique(classification, return_inverse=True)
# y = np.unique(classification, return_inverse=True)

# Split data into training and testing sets
x_train = X[:trainTotal]
x_test = X[trainTotal:]
y_train = y[:trainTotal]
y_test = y[trainTotal:]

from sklearn import preprocessing

# Standardize the features
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Making Fastball Model

In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
# !pip install -U xgboost

names = [
    "Decision Tree",
    "Random Forest",
    "XGBoost"
]

classifiers = [
    # DecisionTreeClassifier(max_depth=50),
    #DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=None, n_estimators=130)
    # XGBClassifier(device='gpu', eta=100)
]

In [ ]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['not Whiff, yes Whiff']
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92     48264
           1       0.90      0.70      0.79     20283

    accuracy                           0.89     68547
   macro avg       0.89      0.83      0.85     68547
weighted avg       0.89      0.89      0.88     68547



In [ ]:
print(accuracies)

[0.8878725545975754]


In [ ]:
print(accuracies)

[0.8870118312982334]


### Export Fastball Model

In [ ]:
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+')

# export the trained model to disk
joblib.dump(clf, 'FastballWhiffModel.joblib')

['FastballWhiffModel.joblib']

## Making Breaking Ball Model

In [ ]:
breaking_minority_class = finalBreaking[finalBreaking['isWhiff'] == 1]
breaking_majority_class = finalBreaking[finalBreaking['isWhiff'] == 0]

print("majority class is: " + str(len(breaking_majority_class)) + " (" + str(100*len(breaking_majority_class)/(len(breaking_majority_class)+len(breaking_minority_class))) + ")")
print("minority class is: " + str(len(breaking_minority_class)) + " (" + str(100*len(breaking_minority_class)/(len(breaking_majority_class)+len(breaking_minority_class))) + ")")

print("total is: " + str(len(breaking_majority_class)+len(breaking_minority_class)))

majority class is: 154299 (66.81461530467315)
minority class is: 76637 (33.185384695326846)
total is: 230936


In [ ]:
# resamples the class
from sklearn.utils import resample
majority_class_downsampled = resample(breaking_majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=len(breaking_minority_class)*2,  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results
minority_class_upsampled = resample(breaking_minority_class,
                                      replace=True,  # sample without replacement
                                      n_samples=int(len(breaking_minority_class)*1.1),  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results

# Combine minority class and downsampled majority class
balanced_df = pd.concat([minority_class_upsampled, majority_class_downsampled])




In [ ]:
# Shuffle the data to avoid order effects
# figure why this don't work
balanced_df = balanced_df.sample(frac=1, random_state=42)


In [ ]:
rel_data = balanced_df[['isWhiff', 'PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'SpeedDrop', 'pfxz', 'VertApprAngle', 'HorzApprAngle']].to_numpy()


classification = balanced_df['isWhiff'].to_numpy()

In [ ]:
minority = balanced_df[balanced_df['isWhiff'] == 1]
majority = balanced_df[balanced_df['isWhiff'] != 1]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

sampledTotal = len(majority) + len(minority)
trainTotal = int(0.8*sampledTotal)

majority class is: 153274
minority class is: 84300
total is: 237574


In [ ]:
# Split data into features and target
X = balanced_df.drop(['isWhiff'], axis=1)
y = balanced_df['isWhiff']
# x = np.unique(classification, return_inverse=True)
# y = np.unique(classification, return_inverse=True)

# Split data into training and testing sets
x_train = X[:trainTotal]
x_test = X[trainTotal:]
y_train = y[:trainTotal]
y_test = y[trainTotal:]

from sklearn import preprocessing

# Standardize the features
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Making Breaking Model

In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
# !pip install -U xgboost

names = [
    "Decision Tree",
    "Random Forest",
    "XGBoost"
]

classifiers = [
    # DecisionTreeClassifier(max_depth=50),
    #DecisionTreeClassifier(max_depth=5),
    # XGBClassifier(device='gpu', eta=100),
    RandomForestClassifier(max_depth=None, n_estimators=100)
]

In [ ]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  print(metrics.accuracy_score(y_test, y_pred))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))


0.8769230769230769


In [ ]:
from sklearn.metrics import classification_report
target_names = ['not Whiff, yes Whiff']
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     30666
           1       0.87      0.76      0.81     16849

    accuracy                           0.88     47515
   macro avg       0.88      0.85      0.86     47515
weighted avg       0.88      0.88      0.87     47515



In [ ]:
print(accuracies)

[0.8755550878669893, 0.8770493528359465, 0.8755550878669893, 0.8773860886035989]


In [ ]:
print(accuracies)

[0.8769230769230769]


### Export Breaking Model

In [ ]:
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+')

# export the trained model to disk
joblib.dump(clf, 'BreakingWhiffModel.joblib')

['BreakingWhiffModel.joblib']

## Making Offspeed Model

In [ ]:
offspeed_minority_class = finalOffspeed[finalOffspeed['isWhiff'] == 1]
offspeed_majority_class = finalOffspeed[finalOffspeed['isWhiff'] == 0]

print("majority class is: " + str(len(offspeed_majority_class)) + " (" + str(100*len(offspeed_majority_class)/(len(offspeed_majority_class)+len(offspeed_minority_class))) + ")")
print("minority class is: " + str(len(offspeed_minority_class)) + " (" + str(100*len(offspeed_minority_class)/(len(offspeed_majority_class)+len(offspeed_minority_class))) + ")")

print("total is: " + str(len(offspeed_majority_class)+len(offspeed_minority_class)))

majority class is: 65308 (66.55524529686322)
minority class is: 32818 (33.44475470313678)
total is: 98126


In [ ]:
# resamples the class
from sklearn.utils import resample
majority_class_downsampled = resample(offspeed_majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=int(len(offspeed_minority_class)*1.9),  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results
minority_class_upsampled = resample(offspeed_minority_class,
                                      replace=True,  # sample without replacement
                                      n_samples=int(len(offspeed_minority_class)*1.1),  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results

# Combine minority class and downsampled majority class
offspeed_balanced_df = pd.concat([minority_class_upsampled, majority_class_downsampled])




In [ ]:
# Shuffle the data to avoid order effects
# figure why this don't work
offspeed_balanced_df = offspeed_balanced_df.sample(frac=1, random_state=42)


In [ ]:
rel_data = offspeed_balanced_df[['isWhiff', 'PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'SpeedDrop', 'pfxz', 'VertApprAngle', 'HorzApprAngle']].to_numpy()
classification = offspeed_balanced_df['isWhiff'].to_numpy()

In [ ]:
minority = offspeed_balanced_df[offspeed_balanced_df['isWhiff'] == 1]
majority = offspeed_balanced_df[offspeed_balanced_df['isWhiff'] != 1]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

sampledTotal = len(majority) + len(minority)
trainTotal = int(0.8*sampledTotal)

majority class is: 62354
minority class is: 36099
total is: 98453


In [ ]:
# Split data into features and target
X = offspeed_balanced_df.drop(['isWhiff'], axis=1)
y = offspeed_balanced_df['isWhiff']
# x = np.unique(classification, return_inverse=True)
# y = np.unique(classification, return_inverse=True)

# Split data into training and testing sets
x_train = X[:trainTotal]
x_test = X[trainTotal:]
y_train = y[:trainTotal]
y_test = y[trainTotal:]

from sklearn import preprocessing

# Standardize the features
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Building Offspeed Model

In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
# !pip install -U xgboost

names = [
    "Decision Tree",
    "Random Forest",
    "XGBoost"
]

classifiers = [
    # DecisionTreeClassifier(max_depth=50),
    #DecisionTreeClassifier(max_depth=5),
    # XGBClassifier(device='gpu', eta=100),
    RandomForestClassifier(max_depth=None, n_estimators=100)
]

In [ ]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  print(metrics.accuracy_score(y_test, y_pred))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))


0.8483571174648317


In [ ]:
from sklearn.metrics import classification_report
target_names = ['not Whiff, yes Whiff']
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.88     12365
           1       0.85      0.71      0.78      7326

    accuracy                           0.85     19691
   macro avg       0.85      0.82      0.83     19691
weighted avg       0.85      0.85      0.84     19691



In [ ]:
print(accuracies)

[0.8755550878669893, 0.8770493528359465, 0.8755550878669893, 0.8773860886035989]


In [ ]:
print(accuracies)

[0.8483571174648317]


### Export Breaking Model

In [ ]:
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+')

# export the trained model to disk
joblib.dump(clf, 'OffspeedWhiffModel.joblib')

['OffspeedWhiffModel.joblib']